This is meant to be a whirlwind tour of Sequence Modeling Language or SMoL, a domain specific probabalistic programming language for sequences over symbols. SMoL is currently an embedded language in Haskell, this document was generated from a Haskell Jupyter Notebook, but Haskell knowledge shouldn't be necessary.

First, I'll show how to build up distributions of sequences with SMoL by composing smaller pieces.
Then I'll demonstrate two types of inference from sequence emission data: decoding and parameter inference. Decoding allows the user to make queries on the generative process for the data in terms of the model's branching variables. Parameter inference can infer the posterior parameters in a sequence model, although this is limited at this point.

# Hello World

In [27]:
-- Some imports we need (this is a comment)
import SMoL
import SMoL.Inference.SHMM
import SMoL.Tags.Utils
import Control.Monad

Let's start with the simplest distribution over non-empty sequences: the distribution that always returns a singleton sequence.

In [3]:
-- The simplest model, besides the empty model
simplest = symbol 'H'

`simplest` is a SMoL expression for this distribution. We can compile it and sample from it:

In [4]:
printSamples 10 (compileSMoL simplest)

"H"
"H"
"H"
"H"
"H"
"H"
"H"
"H"
"H"
"H"

Since `simplest` is deterministic, so we'll always get 'H'.

Almost as simple:

In [5]:
-- Just multiple symbols in a row
elloWorld = symbols "ello world!"

In [6]:
printSamples 10 (compileSMoL elloWorld)

"ello world!"
"ello world!"
"ello world!"
"ello world!"
"ello world!"
"ello world!"
"ello world!"
"ello world!"
"ello world!"
"ello world!"

`elloWorld` is still deterministic like `simplest`, but now the sequence has multiple symbols.

We build up more complex distributions over sequences by composing simpler ones. For example, `andThen` is a function that composes two sequence distributions by concatentating their consistituants:

In [7]:
-- We can use other models as parts
helloWorld = andThen simplest elloWorld

In [8]:
printSamples 10 (compileSMoL helloWorld)

"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Hello world!"

Since `simplest`, `elloWorld` and `andThen` were deterministic, so is `helloWorld`.

`eitherOr` is another way of composing distributions that is not deterministic. The first argument to `eitherOr` first argument (0.6 in this example) is the probability of sampling from the first distribution rather than the second.

In [0]:
-- Models can be probabilistic!
helloGoodbye =
    andThen 
        (eitherOr 0.6
            (symbols "Hello")
            (symbols "Goodbye, cruel"))
        (symbols " world!")

In [10]:
printSamples 10 (compileSMoL helloGoodbye)

"Hello world!"
"Goodbye, cruel world!"
"Goodbye, cruel world!"
"Goodbye, cruel world!"
"Hello world!"
"Hello world!"
"Hello world!"
"Goodbye, cruel world!"
"Hello world!"
"Hello world!"

`helloGoodbye` now represents the distribution that returns "Hello world!" with 60% probability and "Goodbye, cruel world!" with 40% probability.

# Brief introspection

This section is safe to skip if you're not interested in the Haskell types of the SMoL expressions we're working with.

An uncompiled model is of type `ProbSeq a`, where `a` is the type of the symbol in the sequences.

In [12]:
simplest :: ProbSeq Char

simplest :: ProbSeq Char

`compileSMoL` is a function from uncompiled `ProbSeq` to the matrix representation of the distribution, `MatSeq`.

In [13]:
compileSMoL :: forall s. Eq s => ProbSeq s -> MatSeq s

compileSMoL :: forall s. Eq s => ProbSeq s -> MatSeq s

A compiled model is of type `MatSeq a`, where `a` is the type of the symbol in the sequences.

In [14]:
simplestC = compileSMoL simplest

In [15]:
simplestC :: MatSeq Char

simplestC :: MatSeq Char

If we print the value of an uncompiled value like `simplest` from earlier, we get a SMoL AST expression.

In [17]:
simplest

symbol ('H')

If we print a compiled value, we get the actual matrix form of the distribution as well as some bookkeeping.

In [18]:
simplestC

MatSeq {trans = SM SparseMatrix 2x2
1.0	0.0
0.0	1.0
, stateLabels = [StateLabel {stateLabel = 'H', stateTag = StateTag 0 [], tagSet = fromList []}]}

# More tools

This section is a non-exhaustive list of functions that I've built into SMoL for manipulating and composing sequence distributions.

## `finiteDistRepeat`

Repeat an sequence distribution a random number of times according to a given distribution.

In [19]:
repeatModel = finiteDistRepeat [0, 0.1, 0.4, 0, 0.3, 0.2] (andThen (symbols "la") (eitherOr 0.5 (symbols ", ") (symbols "! ")))

printSamples 10 (compileSMoL repeatModel)

"la, la, "
"la, la! "
"la, la! "
"la! la! la, la, "
"la, "
"la, la! "
"la, la! la, la! la! "
"la, la, "
"la, la, "
"la, la, "

## `finiteDistOver`

Choose a sequence distribution at random according to a given distribution. Generalizes `eitherOr`

In [20]:
branchModel = andThen (symbols "Today, I like ") $
    finiteDistOver [
        (symbols "bananas.", 0.4)
      , (symbols "apples.", 0.4)
      , (symbols "grapes.", 0.2)
    ]
printSamples 10 (compileSMoL branchModel)

"Today, I like apples."
"Today, I like bananas."
"Today, I like apples."
"Today, I like apples."
"Today, I like bananas."
"Today, I like apples."
"Today, I like bananas."
"Today, I like apples."
"Today, I like apples."
"Today, I like apples."

## `skip`

`skip n` is a special symbol that, when emitted, skips the next `n` symbols. `skip 0` doesn't do anything.

`possibly`, also shown below, emits nothing with the given probability.

In [21]:
skipModel = andThen (possibly 0.5 (skip 3)) (symbols "do i eat fruit")
printSamples 10 (compileSMoL skipModel)

"do i eat fruit"
"i eat fruit"
"do i eat fruit"
"do i eat fruit"
"i eat fruit"
"i eat fruit"
"do i eat fruit"
"i eat fruit"
"do i eat fruit"
"do i eat fruit"

## `skipDist`

Insert a `skip n`, where `n` is drawn from a given distribution, before each symbol of the given sequence distribution.

In [22]:
skipDistModel = skipDist [0.2, 0.5, 0.2, 0.1] (symbols "do i eat fruit")
printSamples 10 (compileSMoL skipDistModel)

"di eatfui"
"doieatt fruit"
"doo  t ruutt"
"dd ii e uitt"
"oo i aatfrri"
"doi  t uiit"
"o eaa ffrrrit"
"o   ea fruitt"
"d ieeaat  fritt"
"o eat ffrruii"

## `collapse`

Transform each constituant sequence in a given distribution to the sequence of sliding windows (or De Bruijn graph nodes) of a given length.

In [23]:
-- Read: collapse 3 (symbols "do i eat fruit")
collapseModel = collapse undefined (foldl1 (++)) 3 (symbols (map (:[]) "do i eat fruit"))
printSamples 10 (compileSMoL collapseModel)

["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]
["do ","o i"," i ","i e"," ea","eat","at ","t f"," fr","fru","rui","uit"]

# Example application: MinION Sequencer

SMoL was originally motivated by the problem of genotyping DNA given the singal from the Oxford Nanopore Technologies MinION Sequencer. Using SMoL, I can write a very concise and expressive solution to this decoding problem.

DNA is fed into the MinION machine, and a signal is read out. The goal is to be able to use prior information about the input DNA, represented by a SMoL expression, and transform it to a signal-level model, another SMoL expression. To accomplish this, we encode the domain-specific knowledge about how the MinION works - we can encapsulate this knowledge here, and future users can use it.

The MinION signal transformation takes a sliding window over the DNA sequence, with random skips. In SMoL, this is equivalent to a `skipDist` and a `collapse`.

In [24]:
type NT = Char

k = 4

ntSymbols :: [NT] -> ProbSeq [NT]
ntSymbols = symbols . map (:[])

-- Here is the function we want: map from DNA level models to signal level models.
minion :: ProbSeq [NT] -> ProbSeq [NT]
minion = skipDist [0.4, 0.3, 0.2, 0.1]
       . collapse undefined (foldl1 (++)) k


Below is an example of how the MinION transfors a DNA sequence. In this case, we know the DNA sequence for sure, and we take 10 samples.

In [25]:
printSamples 10 . compileSMoL $ minion (ntSymbols "ACGTACACGTATGAC")

["CGTA","ACAC","CACG","CACG","ACGT","CGTA","ATGA","ATGA","ATGA","ATGA","ATGA","TGAC"]
["ACGT","ACGT","ACGT","GTAC","GTAC","GTAC","GTAC","TACA","ACAC","ACAC","ACAC","ACGT","ACGT","ACGT","GTAT","TATG","TGAC","TGAC","TGAC","TGAC"]
["CGTA","CGTA","ACAC","ACAC","ACAC","ACAC","ACGT","CGTA","CGTA","CGTA","GTAT","TATG","TGAC","TGAC","TGAC","TGAC"]
["ACGT","ACGT","CGTA","CGTA","GTAC","GTAC","TACA","ACAC","ACGT","ACGT","GTAT","TATG","ATGA","TGAC"]
["GTAC","TACA","CACG","CGTA","CGTA","CGTA","ATGA","ATGA","ATGA"]
["ACGT","ACGT","GTAC","GTAC","ACAC","ACGT","CGTA","CGTA","ATGA"]
["GTAC","ACAC","CACG","ACGT","ACGT","CGTA","GTAT","GTAT","GTAT","TATG","ATGA"]
["CGTA","TACA","ACAC","ACAC","ACAC","ACGT","ACGT","ACGT","CGTA","GTAT","GTAT","GTAT","ATGA","ATGA","TGAC","TGAC"]
["ACGT","GTAC","ACAC","ACAC","ACAC","CACG","CACG","ACGT","GTAT","TATG","TATG","ATGA","TGAC"]
["GTAC","GTAC","TACA","TACA","TACA","TACA","TACA","TACA","ACAC","CACG","CACG","ACGT","ACGT","GTAT","TATG","ATGA","ATGA","TGAC"]

You can get the idea from the samples - MinION events are 4-wide sliding windows over the input sequence, with random skips and random stalls. The MinION machine's signal is translated into a distribution over these 4-tuples at each time point, which is the matrix passed to SMoL.

# Inference: Decoding

In [0]:
So far, we've defined sequence distributions and sampled from them. We can also do two kinds of inference on sequence data. Let's start with decoding.

We can take a SMoL expression defining our model and some data describing a sequence of distributions over symbols, and we can calculate high-level queries on the process that generated the data.

Below, I define a SMoL model and keep references to 'decision' points in the generative process.

In [28]:
model :: ProbSeq Char
vars :: (Tag Int, Tag Bool)
(model, vars) = runTagGen $ 
  -- The random variable "a" represents the number of times the symbol is repeated
  (ps1, a) <- finiteDistRepeatM [0.8,0.1,0.1] (symbol 'a')
  
  -- The random variable "b" represents the branch taken
  (ps2, b) <- eitherOrM 0.5 ps1 (symbol 'c')
  
  let ps = andThen ps2 (symbol 'd')
  return (ps, (a, b))

-- Sample from the model, just as before
printSamples 10 (compileSMoL model)

"ad"
"ad"
"cd"
"ad"
"ad"
"ad"
"ad"
"cd"
"aaad"
"ad"

As an example, let's generate some sample data from the model.

The matrix that's printed has a row for each point in the sequence, and a column for each symbol.

In [31]:
s <- sample (compileSMoL model)
observations = simulateEmissionsUniform "abcd" 0.5 s

print s
mapM_ print . emissions $ observations

"cd"

[0.125,0.125,0.5,0.125]
[0.125,0.125,0.125,0.5]

We can directly take the posterior over the symbols given our model and the data. There are more columns now, because instead of distributions over symbols, we now have distributions over generating states from our model.

In [32]:
posterior = fullPosteriorSHMM observations (compileSMoL model)

mapM_ print posterior

[0.16666666666666666,0.0,0.0,0.0,0.0,0.8333333333333333,0.0]
[0.0,0.0,0.0,0.0,0.0,0.0,1.0]

The posterior matrix above isn't very user friendly, we would ask queries in the language of our generating model.

To this end, there is a second DSL for defining queries. Below, I define a SMoL query that uses the random variables `a` and `b`. I can query the probability of events of one or more variable, as well as conditional probabilities.

In [33]:
query (a, b) = do
  aDist <- tagDist a
  
  -- p(a = 1 | b is first branch)
  anb <- condition1 b id $ event1 a (== 1)
  
  -- p(b is first branch)
  yb <- event1 b (== True)
  -- p(a > 1)
  ya <- event1 a (> 1)
  
  -- Return p(b is first branch)
  return yb

In [34]:
-- The SMoL function runQuery takes a model with variables, a query, some data, and returns the results of the query.
runQuery :: (ProbSeq d, a) -> (a -> Query b) -> Emissions d -> b

runQuery :: forall d a b. (Show d, Ord d) => (ProbSeq d, a) -> (a -> Query b) -> Emissions d -> b

In [0]:
runQuery (model, vars) query observations

0.01

# Inference: Branch index posteriors

SMoL can also do posterior inference over the branch indices in a sequence distribution. In this section, I use a MinION inference problem as an example. I will define a true model and a prior model, and show that the prior model learns the true model from data. This example performs Short Tandem Repeat (STR) allele inference.

In [38]:
strSegment = ntSymbols "ACT"

-- This function constructs a SMoL MinION model given a sub-model to define the center region.
strProblem strModel = minion $ series [
      ntSymbols "ACGTACACGTATGAC"
    , strModel
    , ntSymbols "TACCAGTTGACAGAT"
    ]

In [39]:
-- The true model repeats the "ACT" sequence exactly 3 times.
strTruth = strProblem (repeatSequence 3 strSegment)
strTruthC = compileSMoL strTruth

In [40]:
-- This function constructs a model given a prior distribution over the number of "ACT" repeats.
-- The parameter is what we're going to infer.
strModel :: [Prob] -> (ProbSeq [NT], Tag Int)
strModel prior = runTagGen $ do
    (ps, repeatVar) <- finiteDistRepeatM prior strSegment
    return (strProblem ps, repeatVar)

-- Define a query that just returns the infered parameter to the model.
strQuery :: Tag Int -> Query [Prob]
strQuery repeatVar = do
    posteriorDist <- tagDist repeatVar
    let (Just distList) = sequence (map (flip Map.lookup posteriorDist) [1..4])
    return distList

In [41]:
-- This expression randomly generates a sample from the true distribution.
simulateSTR :: IO (Emissions [NT])
simulateSTR = do
    s <- sample strTruthC
    return (simulateEmissionsUniform (minionSymbols k) 0.5 s)

In [42]:
sim1 <- simulateSTR

sim1 :: Emissions [NT]

In [44]:
-- This SMoL function takes as input:
--    1. A function from the parameter to a model with random variables
--    2. A function from random variables to a SMoL query
--    3. A first, prior setting for the parameter
--    4. A set of sequence data matricies to learn from
-- And returns as output the final parameter.

updates :: (a -> (ProbSeq b, c)) -> (c -> Query a) -> a -> [Emissions b] -> a

updates :: forall b a c. (Show b, Ord b) => (a -> (ProbSeq b, c)) -> (c -> Query a) -> a -> [Emissions b] -> a

In [45]:
-- Get 5 simulated data matricies
simulations <- replicateM 5 simulateSTR

-- Run inference
updates strModel strQuery [0.25, 0.25, 0.25, 0.25] simulations

[1.800368398168107e-11,2.2899140151477957e-2,0.8986217921194574,7.847906771106108e-2]

Our model, given five noisy examples, assigned 90% posterior probability to the true model, 3 repeats of "ACT".